In [1]:
import pandas as pd
import numpy as np
df1=pd.read_csv('F:/0Sem 7/ML Lab/amazon food review dataset/Reviews.csv')
df1.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
score=df1.values[:,6]
text=df1.values[:,9]
reviews=np.vstack((score,text)).T
print(score.shape, text.shape, reviews.shape)

(568454,) (568454,) (568454, 2)


In [3]:
p=0
n=0
for i in range(reviews.shape[0]):
    if reviews[i,0] > 3:
        reviews[i,0]=0 #positive review
        p=p+1
    else:
        reviews[i,0]=1 #negative review
        n=n+1

In [4]:
reviews = reviews[reviews[:,0].argsort()]   #sort by 1st column

In [5]:
train=[]
for i in range(5000):
    train.append(reviews[i])
    
for i in range(443777,443777+5000):
    train.append(reviews[i])
    
train=np.asarray(train)
train.shape

(10000, 2)

In [6]:
my_reviews1=np.array([0,'This is a very good product. I am very happy with this item.'])
my_reviews2=np.array([1,'The product is very bad. I am very unsatisfied with the appearance.'])
my_reviews3=np.array([0,'It was one if the best items i have purchased. Very good.'])
my_reviews4=np.array([0,'All members of my family enjoyed the item. It is well thought.'])
my_reviews5=np.array([1,'Extremely poor quality. I hated the item and so did my brothers.'])

#train=np.vstack((train,my_reviews1))
#train=np.vstack((train,my_reviews2))
#train=np.vstack((train,my_reviews3))
#train=np.vstack((train,my_reviews4))
#train=np.vstack((train,my_reviews5))

In [7]:
import random as r
test=[]
for i in range(2000):
    index=r.randint(0,reviews.shape[0])
    test.append(reviews[index])
    
test=np.asarray(test)
test.shape

(2000, 2)

In [8]:
#test=np.vstack((test,my_reviews1))
#test=np.vstack((test,my_reviews2))
#test=np.vstack((test,my_reviews3))
#test=np.vstack((test,my_reviews4))
#test=np.vstack((test,my_reviews5))

In [9]:
train_all_words=[]
for i in range(train.shape[0]):
    train_all_words.append(train[i,1].split())
    
train_all_words = [item for sublist in train_all_words for item in sublist]

test_all_words=[]
for i in range(test.shape[0]):
    test_all_words.append(test[i,1].split())
    
test_all_words = [item for sublist in test_all_words for item in sublist]

from collections import Counter
def common_words(words, number_of_words, reverse=False):
    counter = Counter(words)
    return sorted(counter, key = counter.get, reverse=reverse)[:number_of_words]

train_least_common=common_words(train_all_words,200)
train_most_common=common_words(train_all_words,200,reverse=True)

test_least_common=common_words(test_all_words,200)
test_most_common=common_words(test_all_words,200,reverse=True)

In [10]:
for i in range(train.shape[0]):
    train[i,1]=train[i,1].split()
    
for i in range(test.shape[0]):
    test[i,1]=test[i,1].split()

In [11]:
for i in range(train.shape[0]):
    for item in train_most_common:
        if item in train[i,1]:
            train[i,1].remove(item)
    for item in train_least_common:
        if item in train[i,1]:
            train[i,1].remove(item)
            
for i in range(test.shape[0]):
    for item in test_most_common:
        if item in test[i,1]:
            test[i,1].remove(item)
    for item in test_least_common:
        if item in test[i,1]:
            test[i,1].remove(item)

In [12]:
for i in range(train.shape[0]):
    train[i,1]=" ".join(train[i,1])
    
for i in range(test.shape[0]):
    test[i,1]=" ".join(test[i,1])

In [13]:
train_docs=[]
test_docs=[]
train_label=[]
test_label=[]
for i in range(train.shape[0]):
    train_docs.append(train[i,1])
    train_label.append(train[i,0])
    
for i in range(test.shape[0]):
    test_docs.append(test[i,1])
    test_label.append(test[i,0])

In [14]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(train_docs)
vocab_size = len(t.word_index) + 1
#integer encode the documents
encoded_docs = t.texts_to_sequences(train_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#print(padded_docs)
#load the whole embedding into memory

embeddings_index = dict()
f = open('F:/0Sem 7/ML Lab/glove/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

#print('Loaded %s word vectors.' % len(embeddings_index))

#create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Using TensorFlow backend.


In [15]:
embedding_matrix.shape

(19814, 100)

In [16]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#define model
model = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(e)
model.add(layers.Conv1D(32,4, activation='relu'))
model.add(layers.Dropout(rate=0.8))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.LSTM(64, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [17]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1981400   
_________________________________________________________________
conv1d (Conv1D)              (None, 97, 32)            12832     
_________________________________________________________________
dropout (Dropout)            (None, 97, 32)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 48, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8

In [18]:
for layer in model.layers:
    print(layer.output_shape)

(None, 100, 100)
(None, 97, 32)
(None, 97, 32)
(None, 48, 32)
(None, 64)
(None, 64)
(None, 128)
(None, 1)


In [19]:
train_label=np.asarray(train_label)
train_label.shape

(10000,)

In [20]:
padded_docs.shape

(10000, 100)

In [21]:
# prepare tokenizer
t1 = Tokenizer()
t1.fit_on_texts(test_docs)
vocab_size1 = len(t1.word_index) + 1
#integer encode the documents
encoded_docs1 = t1.texts_to_sequences(test_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length1 = 100
padded_docs1 = pad_sequences(encoded_docs1, maxlen=max_length1, padding='post')
test_label=np.asarray(test_label)
#print(padded_docs)

In [22]:
# fit the model
history=model.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=30, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/30
10000/10000 [==============================] - 62s 6ms/sample - loss: 0.6912 - accuracy: 0.5325 - val_loss: 0.6522 - val_accuracy: 0.7740
Epoch 2/30
10000/10000 [==============================] - 23s 2ms/sample - loss: 0.6881 - accuracy: 0.5522 - val_loss: 0.6890 - val_accuracy: 0.6485
Epoch 3/30
10000/10000 [==============================] - 22s 2ms/sample - loss: 0.6915 - accuracy: 0.5364 - val_loss: 0.6508 - val_accuracy: 0.7190
Epoch 4/30
10000/10000 [==============================] - 22s 2ms/sample - loss: 0.6928 - accuracy: 0.5420 - val_loss: 0.6545 - val_accuracy: 0.6625
Epoch 5/30
10000/10000 [==============================] - 22s 2ms/sample - loss: 0.7992 - accuracy: 0.5492 - val_loss: 0.6613 - val_accuracy: 0.6745
Epoch 6/30
10000/10000 [==============================] - 21s 2ms/sample - loss: 0.6900 - accuracy: 0.5370 - val_loss: 0.6607 - val_accuracy: 0.6500
Epoch 7/30
10000/10000 [==============================] -

In [23]:
"2nd LSTM Model"

'2nd LSTM Model'

In [24]:
#define model
model1 = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model1.add(e)
model1.add(layers.Conv1D(32,4, activation='relu'))
model1.add(layers.Dropout(rate=0.8))
model1.add(layers.MaxPooling1D(pool_size=2))
model1.add(layers.LSTM(64, activation='tanh', return_sequences=True))
model1.add(layers.LSTM(128, activation='tanh', return_sequences=False))
model1.add(layers.Flatten())
model1.add(layers.Dense(256, activation ='tanh'))
model1.add(layers.Dense(1, activation='sigmoid'))

# compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1981400   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 97, 32)            12832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 97, 32)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 48, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 48, 64)            24832     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               98816     
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)              

In [25]:
# fit the model
history1=model1.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=30, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/30
10000/10000 [==============================] - 12s 1ms/sample - loss: 0.6878 - accuracy: 0.5520 - val_loss: 0.6908 - val_accuracy: 0.6540
Epoch 2/30
10000/10000 [==============================] - 6s 631us/sample - loss: 0.6151 - accuracy: 0.6680 - val_loss: 0.7830 - val_accuracy: 0.5305
Epoch 3/30
10000/10000 [==============================] - 6s 633us/sample - loss: 0.5735 - accuracy: 0.7066 - val_loss: 0.8322 - val_accuracy: 0.5165
Epoch 4/30
10000/10000 [==============================] - 6s 619us/sample - loss: 0.5440 - accuracy: 0.7311 - val_loss: 0.7899 - val_accuracy: 0.4880
Epoch 5/30
10000/10000 [==============================] - 6s 634us/sample - loss: 0.5279 - accuracy: 0.7389 - val_loss: 0.7094 - val_accuracy: 0.6235
Epoch 6/30
10000/10000 [==============================] - 6s 618us/sample - loss: 0.5151 - accuracy: 0.7529 - val_loss: 0.8374 - val_accuracy: 0.4885
Epoch 7/30
10000/10000 [============================

In [26]:
"3rd LSTM Model"

'3rd LSTM Model'

In [27]:
#define model
model2 = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model2.add(e)
model2.add(layers.Conv1D(32,4, activation='relu'))
model2.add(layers.Dropout(rate=0.8))
model2.add(layers.MaxPooling1D(pool_size=2))
model2.add(layers.LSTM(64, activation='relu', return_sequences=True))
model2.add(layers.LSTM(128, activation='relu', return_sequences=False))
model2.add(layers.Flatten())
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

# compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model2.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1981400   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 97, 32)            12832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 97, 32)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 48, 32)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 48, 64)            24832     
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               98816     
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)              

In [28]:
# fit the model
history2=model2.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=30, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/30
10000/10000 [==============================] - 46s 5ms/sample - loss: 0.6967 - accuracy: 0.5406 - val_loss: 0.6410 - val_accuracy: 0.7230
Epoch 2/30
10000/10000 [==============================] - 41s 4ms/sample - loss: 0.7015 - accuracy: 0.5485 - val_loss: 0.6998 - val_accuracy: 0.5755
Epoch 3/30
10000/10000 [==============================] - 41s 4ms/sample - loss: 0.6904 - accuracy: 0.5519 - val_loss: 0.7032 - val_accuracy: 0.4985
Epoch 4/30
10000/10000 [==============================] - 41s 4ms/sample - loss: 0.6826 - accuracy: 0.5689 - val_loss: 0.6395 - val_accuracy: 0.6165
Epoch 5/30
10000/10000 [==============================] - 41s 4ms/sample - loss: 0.6890 - accuracy: 0.5430 - val_loss: 0.7145 - val_accuracy: 0.2090
Epoch 6/30
10000/10000 [==============================] - 41s 4ms/sample - loss: 0.6942 - accuracy: 0.5295 - val_loss: 0.6895 - val_accuracy: 0.6690
Epoch 7/30
10000/10000 [==============================] -

In [29]:
#define model
model3 = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model3.add(e)
model3.add(layers.Conv1D(32,4, activation='relu'))
model3.add(layers.Dropout(rate=0.8))
model3.add(layers.MaxPooling1D(pool_size=2))
model3.add(layers.GRU(64, activation='relu', return_sequences=True))
model3.add(layers.GRU(128, activation='relu', return_sequences=False))
model3.add(layers.Flatten())
model3.add(layers.Dense(256, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))


# compile the model
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model3.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          1981400   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 97, 32)            12832     
_________________________________________________________________
dropout_3 (Dropout)          (None, 97, 32)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 48, 32)            0         
_________________________________________________________________
gru (GRU)                    (None, 48, 64)            18816     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               74496     
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)              

In [30]:
# fit the model
history3=model3.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=30, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/30
10000/10000 [==============================] - 56s 6ms/sample - loss: 0.6928 - accuracy: 0.5148 - val_loss: 0.6932 - val_accuracy: 0.7275
Epoch 2/30
10000/10000 [==============================] - 51s 5ms/sample - loss: 0.6914 - accuracy: 0.5185 - val_loss: 0.6735 - val_accuracy: 0.7305
Epoch 3/30
10000/10000 [==============================] - 51s 5ms/sample - loss: 0.6904 - accuracy: 0.5245 - val_loss: 0.6689 - val_accuracy: 0.7400
Epoch 4/30
10000/10000 [==============================] - 50s 5ms/sample - loss: 0.6384 - accuracy: 0.6385 - val_loss: 0.5701 - val_accuracy: 0.7115
Epoch 5/30
10000/10000 [==============================] - 51s 5ms/sample - loss: 0.5677 - accuracy: 0.7118 - val_loss: 0.6060 - val_accuracy: 0.6780
Epoch 6/30
10000/10000 [==============================] - 51s 5ms/sample - loss: 0.5398 - accuracy: 0.7293 - val_loss: 0.5801 - val_accuracy: 0.7170
Epoch 7/30
10000/10000 [==============================] -

In [31]:
#define model
model4 = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model4.add(e)
model4.add(layers.Conv1D(32,4, activation='relu'))
model4.add(layers.Dropout(rate=0.8))
model4.add(layers.MaxPooling1D(pool_size=2))
model4.add(layers.Bidirectional(layers.LSTM(64, activation='relu', return_sequences=True)))
model4.add(layers.Bidirectional(layers.LSTM(128, activation='relu', return_sequences=False)))
model4.add(layers.Flatten())
model4.add(layers.Dense(256,activation='relu'))
model4.add(layers.Dense(1, activation='sigmoid'))

# compile the model
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model4.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          1981400   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 97, 32)            12832     
_________________________________________________________________
dropout_4 (Dropout)          (None, 97, 32)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 48, 32)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 128)           49664     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
flatten_4 (Flatten)          (None, 256)              

In [32]:
# fit the model
history4=model4.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=30, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/30
10000/10000 [==============================] - 75s 7ms/sample - loss: 1.7876 - accuracy: 0.5706 - val_loss: 0.6549 - val_accuracy: 0.7920
Epoch 2/30
10000/10000 [==============================] - 68s 7ms/sample - loss: 0.7191 - accuracy: 0.5629 - val_loss: 0.6773 - val_accuracy: 0.7000
Epoch 3/30
10000/10000 [==============================] - 68s 7ms/sample - loss: 20.9758 - accuracy: 0.5716 - val_loss: 0.6079 - val_accuracy: 0.7915
Epoch 4/30
10000/10000 [==============================] - 68s 7ms/sample - loss: 0.6659 - accuracy: 0.5872 - val_loss: 0.6197 - val_accuracy: 0.7745
Epoch 5/30
10000/10000 [==============================] - 68s 7ms/sample - loss: 0.6546 - accuracy: 0.6074 - val_loss: 0.6584 - val_accuracy: 0.6960
Epoch 6/30
10000/10000 [==============================] - 67s 7ms/sample - loss: 0.6522 - accuracy: 0.6142 - val_loss: 0.6287 - val_accuracy: 0.7610
Epoch 7/30
10000/10000 [==============================] 